In [11]:
from openai import OpenAI
from getpass import getpass
import json

In [2]:
# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [155]:
with open('data/check_card_info-ver-3.json', 'r', encoding='utf-8') as json_file:
    check_card_data = json.load(json_file)

with open('data/credit_card_info-ver-3.json', 'r', encoding='utf-8') as json_file:
    credit_card_data = json.load(json_file)

In [162]:
question = '''체크 카드 데이터에 있는 89번째 카드 이름이 뭐야??'''

def analysis_data(check, question, model):
    '''파일과 질의를 구분한 사용자 정의 함수'''
    prompt = f'체크 카드 데이터 : {check}\nQuestion : {question}'
    response = client.chat.completions.create(model=model,
                                          messages=[{'role':'user','content':prompt}],
                                          max_tokens=500 
                                          )
    
    return response.choices[0].message.content

analysis_data(credit_card_data, question, 'gpt-3.5-turbo')

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 28844 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [122]:
# 유의사항, 기타 항목 제거
for i in range(len(credit_card_data)):
    for k, v in credit_card_data[i].items():
        v['주요 혜택'] = {sub_k: sub_v for sub_k, sub_v in v['주요 혜택'].items() if sub_k not in ['유의사항', '기타']}


In [123]:
# 이미지 파일 명에서 코드 항목만 추출하여 코드를 검색하여 이미지를 찾을 수 있게 수정
import re

for i in range(len(credit_card_data)):
    for k, v in credit_card_data[i].items():
        file_name = v['카드 이미지'].split('\\')[1]  # 파일명 추출

        match = re.search(r'(\d+)[a-zA-Z]', file_name)
        if match:
            extracted_number = match.group(1)  # "2749"
            
            del v['카드 이미지']  # '카드 이미지' 항목 제거
            v['code'] = extracted_number  # 새로운 'code' 항목 생성

In [158]:
# 주요 혜택 카테고리의 대괄호(부연 설명) 제거
for i in range(len(credit_card_data)):
    for k, v in credit_card_data[i].items():
        for sub_k, sub_v in v['주요 혜택'].items():
            v['주요 혜택'][sub_k] = re.sub(r'\[.*?\]\s*', '', sub_v)
        

In [159]:
with open('data/credit_card_info-ver-4.json', 'w', encoding='utf-8') as json_file:
    json.dump(credit_card_data, json_file, ensure_ascii=False, indent=4)

In [160]:
credit_card_data

[{'신한카드 Mr.Life': {'혜택 요약': '공과금 10% 할인, 마트,편의점 10% 할인, 식음료 10% 할인',
   '연회비 상세안내': 'S& 15,000원 해외겸용(VISA) 18,000원 * URS, UPI, S& 브랜드는 로컬과 연회비가 동일하며, 해외 JCB(URS)/은련(UPI)/AMEX(S&) 가맹점에서 사용가능합니다.',
   '주요 혜택': {'공과금': '월납요금(공과금) 10% 할인서비스',
    '편의점': '편의점 10% 할인',
    '병원/약국': '병원/약국업종 10% 할인',
    '생활': '세탁소 업종 10% 할인',
    '온라인쇼핑': ' 인테이크몰 20% 할인',
    '택시': '택시 10% 할인 ',
    '푸드': '식음료 10% 할인',
    '대형마트': '3대 마트 10% 할인',
    '주유소': '4대 정유사 리터당 60원 할인'},
   'code': '13'}},
 {'삼성카드 taptap O': {'혜택 요약': '스타벅스 50% 할인, 대중교통 10% 할인, CGV, 롯데시네마 5,000원 할인',
   '연회비 상세안내': '국내전용 10,000원 (기본 7,000원 + 제휴 3,000원) 해외겸용(Mastercard) 10,000원 (기본 7,000원 + 제휴 3,000원)',
   '주요 혜택': {'선택형': '라이프스타일 패키지 선택',
    '대중교통': '대중교통·택시 10% 결제일할인',
    '통신': '이동통신요금 10% 결제일할인',
    '영화': 'CGV 및 롯데시네마 5,000원 결제일할인',
    '해외': '해외 1.3% 적립'},
   'code': '51'}},
 {'삼성카드 & MILEAGE PLATINUM (스카이패스)': {'혜택 요약': '1,000원당 1마일 기본적립, 국내/해외 택1 옵션형 1마일 특별적립, 인천공항 라운지 무료이용',
   '연회비 상세안내': '국내전용 47,000원 (기본 7,000원 + 제휴 40,00